Com a coluna conteudo resumida

In [ ]:
import pandas as pd
df_final = pd.read_csv("../dados/df_limpo.csv")

Extrair informações importantes de texto

Limite de tokens: Gemini 1.5 Flash suporta até 1 milhão de tokens contextuais, mas é mais rápido e mais barato com textos curtos. Se o documento for muito longo, divida em trechos.

Prompt estruturado = melhor resultado: Use listas, marcadores e perguntas diretas.

Padronize o input: Remova cabeçalhos, assinaturas ou trechos irrelevantes.

Verifique consistência: LLMs podem gerar variações. Se precisar de formato JSON, peça explicitamente.

In [14]:
import os, json
import pandas as pd
from dotenv import load_dotenv
from tqdm.auto import tqdm
import google.generativeai as genai

load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

# Configura o Gemini
genai.configure(api_key=API_KEY)
modelo = genai.GenerativeModel(
    "gemini-1.5-flash",
    generation_config={"temperature": 0.1}  
)



In [15]:
# ---------------------------------------------------------------
# 1. Função de limpeza do JSON
# ---------------------------------------------------------------
def _json_limpo(resposta):
    """Isola o bloco JSON e converte em dict."""
    resposta = resposta.strip()
    bloco = resposta[resposta.find("{"): resposta.rfind("}") + 1]
    return json.loads(bloco)

# ---------------------------------------------------------------
# 2. Prompt fixo + extrair_info
# ---------------------------------------------------------------
_PROMPT = """
Devolva apenas JSON com as chaves e tipos abaixo, usando null onde faltar:

{{
  "valor_multa_reais": float|null,
  "tipo_infracao_concorrencial": string|null,
  "seguiu_nota_tecnica": true|false,
  "houve_condenacao": true|false,
  "tipo_decisao": string|null,
  "houve_acordo": true|false,
  "reincidencia": true|false,
}}

TEXTO:
\"\"\"{texto}\"\"\"
"""

def extrair_info(texto):
    """Executa o prompt no Gemini e devolve dict com 21 campos."""
    resposta = modelo.generate_content(_PROMPT.format(texto=texto))
    return _json_limpo(resposta.text)



In [16]:
a= df_final['resumo'].iloc[0]
extrair_info(a)

{'valor_multa_reais': 324116.21,
 'tipo_infracao_concorrencial': 'cartel de peças automotivas',
 'seguiu_nota_tecnica': True,
 'houve_condenacao': True,
 'tipo_decisao': 'Condenação parcial com imposição de multa pecuniária',
 'houve_acordo': True,
 'reincidencia': False}

In [17]:
a= df_final['conteudo'].iloc[0]
extrair_info(a)

{'valor_multa_reais': 324116.21,
 'tipo_infracao_concorrencial': 'cartel de peças automotivas',
 'seguiu_nota_tecnica': True,
 'houve_condenacao': True,
 'tipo_decisao': 'Condenação parcial com imposição de multa pecuniária',
 'houve_acordo': True,
 'reincidencia': False}

In [18]:
df_final.head(2)

,id,ano_documento,assinaturas,descricao_tipo_documento,decisao_tribunal,setor_economico,id_unidade,data_processo,data_documento,diferenca_dias,mercado_relevante,descricao_especificacao,conteudo,resumo
0,abbdc450-7ff1-4990-9bcc-8ae4386592f0,2024,Outras,Voto Processo Administrativo,vazio,Fabricação de Peças e Acessórios para o Sistem...,110000967.0,2019-09-13,2024-05-23,1714,"pistões de motor, bronzinas, camisas, pinos, b...","- Desmembrado Bernd Brünig, Faustino Luigi Mi...",SEI/CADE - 1390607 - Voto Processo Administrat...,"['\nREPRESENTADOS(AS): BERND BRÜNIG, FAUSTINO ..."
1,b41e8f1f-449e-4d28-a574-18f365d32b6f,2024,Gustavo Augusto Freitas de Lima,Voto Processo Administrativo,condenacao,Gestão e Administração da Propriedade Imobiliária,110000969.0,2015-05-21,2024-05-09,3276,Mercado imobiliário,Influencia de conduta uniforme - COFECI,SEI/CADE - 1384594 - Voto Processo Administrat...,"['\nNOS TERMOS DO VOTO RECORRIDO, ESTE TRIBUNA..."


In [19]:
a= df_final['resumo'].iloc[1]
extrair_info(a)

{'valor_multa_reais': 100000.0,
 'tipo_infracao_concorrencial': 'imposição de cumprimento de tabela de preços',
 'seguiu_nota_tecnica': True,
 'houve_condenacao': True,
 'tipo_decisao': 'manutenção da condenação',
 'houve_acordo': True,
 'reincidencia': False}

In [20]:
a= df_final['conteudo'].iloc[1]
extrair_info(a)

{'valor_multa_reais': 100000.0,
 'tipo_infracao_concorrencial': 'Imposição de tabela de preços mínimos',
 'seguiu_nota_tecnica': True,
 'houve_condenacao': True,
 'tipo_decisao': 'Nego provimento',
 'houve_acordo': True,
 'reincidencia': True}

In [21]:
a= df_final['resumo'].iloc[2]
extrair_info(a)

{'valor_multa_reais': None,
 'tipo_infracao_concorrencial': 'cartel no mercado de revenda de combustíveis',
 'seguiu_nota_tecnica': True,
 'houve_condenacao': False,
 'tipo_decisao': 'arquivamento',
 'houve_acordo': False,
 'reincidencia': False}

In [22]:
!pip install ace_tools

In [23]:
import pandas as pd
import ace_tools as tools

# Define the column names and their descriptions
columns_with_descriptions = {
    "valor_multa_reais": "Valor da multa aplicada em reais (float).",
    "percentual_faturamento": "Multa como percentual do faturamento da empresa (%).",
    "numero_multas": "Quantidade total de multas aplicadas na decisão.",
    "setor_padronizado": "Setor econômico ao qual a empresa pertence (ex: saúde, financeiro).",
    "quantidade_conselheiros": "Número total de conselheiros que votaram.",
    "votos_favoraveis": "Número de votos favoráveis à condenação.",
    "votos_contrarios": "Número de votos contrários à condenação.",
    "voto_vencedor": "Resultado do voto vencedor (ex: condenação, arquivamento).",
    "tipo_infracao_concorrencial": "Tipo da infração (ex: cartel, abuso de posição dominante).",
    "seguiu_nota_tecnica": "Indica se a decisão seguiu a nota técnica (True/False).",
    "tempo_processual_dias": "Tempo total do processo em dias.",
    "houve_condenacao": "Indica se houve condenação na decisão (True/False).",
    "tipo_decisao": "Classificação da decisão (ex: condenação, arquivamento, acordo).",
    "houve_acordo": "Indica se houve acordo como TCC (True/False).",
    "empresa_envolvida": "Nome da(s) empresa(s) envolvida(s).",
    "porte_empresa": "Porte da empresa (ex: micro, pequena, média, grande).",
    "relator_voto": "Nome do conselheiro relator do processo.",
    "ano_decisao": "Ano da decisão final.",
    "reincidencia": "Indica se a empresa é reincidente (True/False).",
    "descricao_fatos_relevantes": "Resumo dos fatos relevantes do caso.",
    "documento_origem": "Link ou identificador do documento de origem."
}

# Create an empty DataFrame with these columns
df = pd.DataFrame(columns=columns_with_descriptions.keys())

tools.display_dataframe_to_user(name="CADE_Analise_Variaveis", dataframe=df)


ModuleNotFoundError: No module named 'ace_tools'